In [70]:
import cv2
import numpy as np
import xml.etree.ElementTree as ET
import os
import glob

In [62]:
def get_objects(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    objects = []
    for obj in root.findall('object'):
        bndbox = obj.find('bndbox')
        xmin = int(bndbox.find('xmin').text)
        ymin = int(bndbox.find('ymin').text)
        xmax = int(bndbox.find('xmax').text)
        ymax = int(bndbox.find('ymax').text)
        objects.append((xmin, ymin, xmax, ymax))
    
    return objects

def put_objects(objects, xml_file):
    head, tail = os.path.split(xml_file)
    filename, _ = os.path.splitext(tail)
    
    root = ET.Element('annotation')
    header = ET.XML('''
                    <root>
                        <folder>ImageDatabase</folder>
                        <filename>{}</filename>
                        <path>{}</path>
                        <source>
                            <database>Unknown</database>
                        </source>
                        <size>
                            <width>300</width>
                            <height>300</height>
                            <depth>3</depth>
                        </size>
                        <segmented>0</segmented>
                    </root>
                    '''.format(filename + '.jpg', os.path.join(head, filename + '.jpg')))
    root.extend(header)
    
    for xmin, ymin, xmax, ymax in objects:
        obj = ET.XML('''
                     <root>
                         <object>
                             <name>car2</name>
                             <pose>Unspecified</pose>
                             <truncated>0</truncated>
                             <difficult>0</difficult>
                             <bndbox>
                                 <xmin>{}</xmin>
                                 <ymin>{}</ymin>
                                 <xmax>{}</xmax>
                                 <ymax>{}</ymax>
                             </bndbox>
                         </object>
                     </root>
                     '''.format(xmin, ymin, xmax, ymax))
        root.extend(obj)
    ET.ElementTree(root).write(xml_file)

In [67]:
def split_image(img_path, xml_path):
    objects = get_objects(xml_path)
    img = cv2.imread(img_path)
    mask = cv2.imread('./img/mask.png', 0)
    img = cv2.bitwise_and(img, img, mask=mask)

    #for xmin, ymin, xmax, ymax in objects:
    #    cv2.rectangle(img, (xmin, ymin), (xmax, ymax), (255, 0, 0), 1)
    
    coords = (
                ((190, 125), (490, 425)),
                ((440, 125), (740, 425)),
                ((690, 125), (990, 425)),
                ((940, 125), (1240, 425)),
                ((0, 350), (300, 650)),
                ((245, 350), (545, 650)),
                ((490, 350), (790, 650)),
                ((735, 350), (1035, 650)),
                ((980, 350), (1280, 650))
    )
    
    #cv2.rectangle(img, coords[0][0], coords[0][1], (0, 0, 255), 1)
    #cv2.rectangle(img, coords[1][0], coords[1][1], (0, 0, 255), 1)
    #cv2.rectangle(img, coords[2][0], coords[2][1], (0, 0, 255), 1)
    #cv2.rectangle(img, coords[3][0], coords[3][1], (0, 0, 255), 1)
    #cv2.rectangle(img, coords[4][0], coords[4][1], (0, 0, 255), 1)
    #cv2.rectangle(img, coords[5][0], coords[5][1], (0, 0, 255), 1)
    #cv2.rectangle(img, coords[6][0], coords[6][1], (0, 0, 255), 1)
    #cv2.rectangle(img, coords[7][0], coords[7][1], (0, 0, 255), 1)
    #cv2.rectangle(img, coords[8][0], coords[8][1], (0, 0, 255), 1)

    head, tail = os.path.split(img_path)
    filename, ext = os.path.splitext(tail)
    
    for i, c in enumerate(coords):
        poi = img[c[0][1]:c[1][1], c[0][0]:c[1][0]]
        poi_img_path = os.path.join(head, 'split',filename + '-' + str(i) + ext)
        poi_xml_path = os.path.join(head, 'split', filename + '-' + str(i) + '.xml')
        poi_objects = []
    
        for xmin, ymin, xmax, ymax in objects:
            if xmin >= c[0][0] and ymin >= c[0][1]:
                if xmax < c[1][0] and ymax < c[1][1]:
                    poi_objects.append((xmin - c[0][0], ymin - c[0][1], xmax - c[0][0], ymax - c[0][1]))
    
        put_objects(poi_objects, poi_xml_path)
        cv2.imwrite(poi_img_path, poi)

In [71]:
path = 'C:\\Users\\szla\\Data\\RND\\ImageDatabase\\pklot\\PUCPR\\DONE\\'

for f in glob.glob(path + '*.jpg'):
    head, tail = os.path.split(f)
    filename, ext = os.path.splitext(tail)
    split_image(os.path.join(head, filename + ext), os.path.join(head, filename + '.xml'))